In [1]:
from txgnn import TxGNN, TxEval, TxData, NewDataHandler
from txgnn.utils import *
from tqdm import tqdm
import time
import argparse

split = "complex_disease"
seed = 1
saving_path = './properly_pre_trained_model_ckpt/'

TxData1 = TxData(data_folder_path = './data/')
# psuedo_label_fname = "psuedo_scores_in_og_dataset.csv"
# pseudo_labels = pd.read_csv(psuedo_label_fname)
# TxData1.prepare_split(split = split, seed = seed, no_kg = False, additional_train=pseudo_labels, soft_pseudo=True)
TxData1.prepare_split(split = split, seed = seed, no_kg = False)

Using GPU: NVIDIA GeForce RTX 2080 Ti
Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
additional 
Done!


In [2]:
TxGNN1 = TxGNN(
        data = TxData1, 
        weight_bias_track = False,
        proj_name = 'TxGNN',
        exp_name = 'TxGNN'
    )
TxGNN1.model_initialize(n_hid = 100, 
                        n_inp = 100, 
                        n_out = 100, 
                        proto = True,
                        proto_num = 3,
                        attention = False,
                        sim_measure = 'all_nodes_profile',
                        bert_measure = 'disease_name',
                        agg_measure = 'rarity',
                        num_walks = 200,
                        walk_mode = 'bit',
                        path_length = 2,
                        LSP_size="partial"
                        )

In [3]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--save_dir', type=str, help="fname in properly_pre_trained_model_ckpt folder") #### Test #### How about without default=None, is this implied? 
# args = parser.parse_args()
# save_dir = f'{saving_path}seed_1_restrained_saveG'

# TxGNN1.pretrain(n_epoch = 1, ## was 2
#                learning_rate = 1e-3,
#                batch_size = 1024, 
#                train_print_per_n = 20)

# TxGNN1.finetune(n_epoch = 500, #---
#                 learning_rate = 5e-4,
#                 train_print_per_n = 5,
#                 valid_per_n = 20,)

# save_dir = f'{saving_path}seed_1_restrained'
# TxGNN1.load_pretrained(save_dir, legacy=True)

save_dir = f'{saving_path}seed_1_restrained_saveG/'
TxGNN1.load_pretrained(save_dir)

G = TxGNN1.G.to(TxGNN1.device)
best_G = TxGNN1.best_G.to(TxGNN1.device)
best_model = TxGNN1.best_model.to(TxGNN1.device)
best_model.eval()
# print(TxGNN1.dd_etypes.device)
# TxGNN1.model = TxGNN1.model.to(TxGNN1.device)
# TxGNN1.best_model = TxGNN1.best_model.to(TxGNN1.device)
# print(TxGNN1.model)
# print("encore")

## Print out accuracies
neg_sampler = Full_Graph_NegSampler(TxGNN1.G, 1, 'fix_dst', TxGNN1.device)
g_train_neg = neg_sampler(TxGNN1.G)
g_valid_pos, g_valid_neg = TxGNN1.g_valid_pos, TxGNN1.g_valid_neg
g_test_pos, g_test_neg = TxGNN1.g_test_pos, TxGNN1.g_test_neg
## parameters: (model, G, train_pos, train_neg, val_pos, val_neg, test_pos, test_neg, device=None)
accuracies_dict = evaluate_accuracy_per_split(best_model, best_G, g_train_neg, g_valid_pos, g_valid_neg, g_test_pos, g_test_neg)
for k, v in accuracies_dict.items():
    print(f"Accuracy for Positive {k} graph is {v[0].item()}")
    print(f"Accuracy for Negative {k} graph is {v[1].item()}")
print_val_test_auprc(best_model, g_valid_pos, g_valid_neg, g_test_pos, g_test_neg, best_G, TxGNN1.dd_etypes, TxGNN1.device)

Checkpoint loaded from path: ./properly_pre_trained_model_ckpt/seed_1_restrained_saveG/
Using stored similarity matrix for ('drug', 'contraindication', 'disease')
Using stored similarity matrix for ('drug', 'indication', 'disease')
Using stored similarity matrix for ('drug', 'contraindication', 'disease')
Using stored similarity matrix for ('drug', 'indication', 'disease')
Using stored similarity matrix for ('drug', 'contraindication', 'disease')
Using stored similarity matrix for ('drug', 'indication', 'disease')
Using stored similarity matrix for ('drug', 'contraindication', 'disease')
Using stored similarity matrix for ('drug', 'indication', 'disease')
Using stored similarity matrix for ('drug', 'contraindication', 'disease')
Using stored similarity matrix for ('drug', 'indication', 'disease')
Using stored similarity matrix for ('drug', 'contraindication', 'disease')
Using stored similarity matrix for ('drug', 'indication', 'disease')
Accuracy for Positive train graph is 0.914647638

In [31]:
TxGNN1.g_pos_pseudo
best_G
# torch.where(best_G.edges(etype=('drug', 'indication', 'disease')))
# torch.where(best_G.in_degrees(etype=('drug', 'indication', 'disease')))
sum(best_G.in_degrees(etype=('drug', 'indication', 'disease')))
sum(TxGNN1.g_pos_pseudo.in_degrees(etype=('drug', 'indication', 'disease')))

tensor(9388, device='cuda:0')

In [92]:
# df_train = TxGNN1.df_train
# df_train[y_id]
disease_keyid = torch.where(best_G.in_degrees(etype=('drug', 'indication', 'disease')))[0]
# disease_queryid = torch.where(TxGNN1.g_pos_pseudo.in_degrees(etype=('drug', 'indication', 'disease')))[0]
disease_queryid = torch.tensor([39], device=torch.device("cuda"))

mask = torch.isin(disease_queryid, disease_keyid)

In [97]:
TxGNN1.g_valid_pos.edges(etype=('drug', 'indication', 'disease'))[1]

tensor([12661, 12661, 12661,  ...,  4301, 15175, 13194], device='cuda:0')

TypeError: len() of a 0-d tensor

In [ ]:
## Why are we seeing so many Falses when training with pseudo labels?
    ## Because not all diseases with specific edge type in eval graph is in Train Graph
    ## This suggests maybe it is beneficial for "complex diseases" to not use any DPM during training? 

## Let's make sure that according to the theory above, we should see True rate of 80% during training with  pseudo labels
    ## How come the above is like nan or 501 when I'm training with pseudo labels? 


In [ ]:
# kg_path = os.path.join('./data', 'kg_directed.csv')
# full_kg_df = pd.read_csv(kg_path)
# train_kg = create_dgl_graph(best_G, best_G).to(TxGNN1.device)
# full_kg = create_dgl_graph(full_kg_d"f, best_G).to(TxGNN1.device)

h, beta_kl_loss, distmult = best_model(best_G, pretrain_mode = False, mode = 'train', return_h_and_kl=True)
# pos_score, out_pos, LS = distmult(best_G, best_G, h, mode=None, pretrain_mode=False, LSP="L2")
_, _, LS = distmult(best_G, best_G, h, mode=None, pretrain_mode=False, LSP="RBF", LSP_size="partial")
# pred_score_pos, out_pos = distmult(g_valid_pos, best_G, h, mode=None, pretrain_mode=False)
# _, _, LS = distmult(self.g_pos_pseudo, self.G, h, mode=None, pretrain_mode=False, pseudo_training=True, LSP=True)

Using stored similarity matrix for ('disease', 'disease_disease', 'disease')
Using stored similarity matrix for ('gene/protein', 'disease_protein', 'disease')
time it took to create the sparse tensor: 1.7757363319396973


In [6]:
# print(LS._nnz())
# print(LS.coalesce()._nnz())
LS_prob = []
for i, v in enumerate(LS):
    softmaxed = F.softmax(v)
    LS_prob.append(softmaxed)
len(LS_prob)

17080

In [7]:

# torch.save(LS_prob, "LSP_full_RBF.pt")
torch.save(LS_prob, "LSP_partial_RBF.pt")

# LS
# LS_target =  torch.load("LSP_partial_cosine.pt")
# LS_target =  LS_prob

In [24]:
# LSP_loss = 0
# for i, (LS_i, LS_target_i) in enumerate(zip(LS, LS_target)):
#     LS_i = LS_i.coalesce()
#     LS_target_i = LS_target_i.coalesce()
#     if len(LS_i.values()) > 0:
#         ## softmax LS_i
#         try:
#             pred_prob = F.softmax(LS_i.values())
#             # target_prob = F.softmax(LS_target_i.values())
#             target_prob = LS_target_i.values()
#             ## compute loss
#             LSP_loss = LSP_loss + F.kl_div(pred_prob.log(), target_prob)
#         except Exception as e:
#             print(i)
#             print(e)
#             # print(LS_i.indices())
#             # print(LS_target_i.indices())
            
#             # print(LS_i.values())    
#             # print(LS_target_i.values())
#     #         print(pred_prob.shape, target_prob.shape)
# LSP_loss

tensor(0., device='cuda:0', grad_fn=<AddBackward0>)

In [35]:
# for i, (LS_i, v2) in enumerate(zip(LS_, temp)):
#     LS_i = LS_i.coalesce()
#     print(i)
#     pred = LS_i.values()
#     pred_prob = F.softmax(pred)
#     print(pred_prob)
#     break

0
tensor([0.0453, 0.0405, 0.0441, 0.0418, 0.0441, 0.0430, 0.0311, 0.0397, 0.0419,
        0.0258, 0.0453, 0.0407, 0.0453, 0.0451, 0.0404, 0.0441, 0.0282, 0.0442,
        0.0390, 0.0455, 0.0451, 0.0441, 0.0447, 0.0239, 0.0272],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [20]:
# for i,v in enumerate(LS_):
#     print(i)
#     print(v)
#     break

# for i,v in enumerate(LS_.indices()):
#     print(i)
#     print(v)
#     break

# for v in LS_.indices():
#     print(v)
#     break

tensor([    0,     0,     0,  ..., 17077, 17079, 17079], device='cuda:0')


In [28]:
# (LS_.coalesce().indices() == LS_.coalesce().indices()).float().mean()

tensor(1., device='cuda:0')

In [29]:
# indices = LS_.indices()
# norm_values = []
# for i in range(17080):
#     mask = indices[0] == i
#     compressed_tensor = LS_.values()[mask]
#     norm_ct = F.softmax(compressed_tensor)
#     norm_values.append(norm_ct)
    
# norm_values = torch.cat(norm_values)
# print(norm_values)

tensor([0.0453, 0.0405, 0.0441,  ..., 0.0111, 0.0115, 0.0123], device='cuda:0',
       grad_fn=<CatBackward0>)


In [43]:
for i in range(100):
    mask = indices[0] == i
    print(norm_values[mask])
    break


tensor([0.0453, 0.0405, 0.0441, 0.0418, 0.0441, 0.0430, 0.0311, 0.0397, 0.0419,
        0.0258, 0.0453, 0.0407, 0.0453, 0.0451, 0.0404, 0.0441, 0.0282, 0.0442,
        0.0390, 0.0455, 0.0451, 0.0441, 0.0447, 0.0239, 0.0272],
       device='cuda:0', grad_fn=<IndexBackward0>)


In [22]:
# mask = LS_.indices()[0] == 333
# LS_target.values()[mask]
# LS_.values()[mask]
# LS_.values().shape

tensor([0.3115, 0.4254, 0.6658, 0.4438, 0.4059, 0.1487, 0.1337, 0.1096],
       device='cuda:0', grad_fn=<IndexBackward0>)

In [30]:
LS_prob = torch.sparse_coo_tensor(indices, norm_values, LS_.size())
LS_prob

tensor(indices=tensor([[    0,     0,     0,  ..., 17079, 17079, 17079],
                       [   67,   188,  6340,  ..., 47585, 89183, 93538]]),
       values=tensor([0.0453, 0.0405, 0.0441,  ..., 0.0111, 0.0115, 0.0123]),
       device='cuda:0', size=(17080, 100001), nnz=284544,
       layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)

In [31]:
(LS_.coalesce().indices() == LS_prob.coalesce().indices()).float().mean()



tensor(1., device='cuda:0')

In [32]:
# device = torch.device("cuda")

# # LS = torch.zeros(n_dis, n_max, len(etypes), device=self.device) ## [N, N, E]
# src = torch.arange(500)
# dst = torch.arange(20, 520)
# values = torch.arange(999, 1499)
# # print(src)
# ex = torch.sparse_coo_tensor(torch.stack([src, dst]), values, (35000, 35000))
# ex2 = torch.sparse_coo_tensor(torch.stack([dst, src]), values, (35000, 35000))
# ex + ex2

tensor(indices=tensor([[  0,   1,   2,  ..., 517, 518, 519],
                       [ 20,  21,  22,  ..., 497, 498, 499]]),
       values=tensor([ 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007,
                      1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016,
                      1017, 1018,  999, 1019, 1000, 1020, 1001, 1021, 1002,
                      1022, 1003, 1023, 1004, 1024, 1005, 1025, 1006, 1026,
                      1007, 1027, 1008, 1028, 1009, 1029, 1010, 1030, 1011,
                      1031, 1012, 1032, 1013, 1033, 1014, 1034, 1015, 1035,
                      1016, 1036, 1017, 1037, 1018, 1038, 1019, 1039, 1020,
                      1040, 1021, 1041, 1022, 1042, 1023, 1043, 1024, 1044,
                      1025, 1045, 1026, 1046, 1027, 1047, 1028, 1048, 1029,
                      1049, 1030, 1050, 1031, 1051, 1032, 1052, 1033, 1053,
                      1034, 1054, 1035, 1055, 1036, 1056, 1037, 1057, 1038,
                      1058, 1039, 1059, 

In [16]:
# LS_nonzero = torch.where(LS == 0, torch.tensor(float("-inf")), LS)
# LSP_prob = torch.clamp(torch.sigmoid(LS_nonzero), min=1e-9)
# LSP_prob
# LSP_prob = [k:torch.clamp(torch.sigmoid(v), min=1e-9) for k,v in enumerate(LS)]
# LSP_norm_prob = {k: v/v.sum() for k,v in LSP_prob.items()}
# LSP_norm_prob

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.76 GiB. GPU 0 has a total capacty of 10.75 GiB of which 236.50 MiB is free. Process 142512 has 10.52 GiB memory in use. Of the allocated memory 8.46 GiB is allocated by PyTorch, and 641.53 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [32]:
import pickle
# with open('LSP.pkl', "rb") as file:
#     LS_dict = pickle.load(file)
# LS_dict
# with open("LSP_allddetype.pkl", "wb") as file:
#     pickle.dump(LSP_norm_prob, file)
# with open("LSP_partialetype.pkl", "wb") as file:
#     pickle.dump(LSP_norm_prob, file)

torch.save(LS_prob, "LSP_full_cosine.pt")

# torch.save(LS_prob, "LSP_partial_cosine.pt")
# loaded = torch.load("LSP_partial_cosine.pt")
# loaded

In [ ]:
data_path = "./data/complex_disease_1/"
# df_train = pd.read_csv(f"{data_path}train.csv")
# df_valid = pd.read_csv(f"{data_path}valid.csv")
# df_test = pd.read_csv(f"{data_path}test.csv")

# df_valid_nocheating = df_valid[(df_valid.relation != "indication") & (df_valid.relation != "contraindication")]
df_test_nocheating = df_test[(df_test.relation != "indication") & (df_test.relation != "contraindication")]

# df_valid_nocheating
kg_no_cheating = pd.concat([df_train, df_valid_nocheating, df_test_nocheating])
kg_no_cheating.to_csv(f"{data_path}/kg_nocheating.csv")

In [ ]:
## Problems
# 1. Does disease pooling mechanism happen during tarining phase too? 
    # Yes. So let's enable dpm during pseudo training with only diseases in training set. 
# 2. How can I store the LS matrix with augmented disease embeddings? 
    # I did this, and pretty sure the scores were matched. But for some reason, validation phase still takes a very long time... why? 

In [ ]:
# save_dir = f'{saving_path}seed_1_normal'
# TxGNN1.load_pretrained(save_dir, legacy=True)
# G = TxGNN1.G.to(TxGNN1.device)
# best_G = TxGNN1.best_G.to(TxGNN1.device)
# best_model = TxGNN1.best_model.to(TxGNN1.device)
# best_model.eval()
# # print(TxGNN1.dd_etypes.device)
# # TxGNN1.model = TxGNN1.model.to(TxGNN1.device)
# # TxGNN1.best_model = TxGNN1.best_model.to(TxGNN1.device)
# # print(TxGNN1.model)
# # print("encore")

# ## Print out accuracies
# neg_sampler = Full_Graph_NegSampler(TxGNN1.G, 1, 'fix_dst', TxGNN1.device)
# g_train_neg = neg_sampler(TxGNN1.G)
# g_valid_pos, g_valid_neg = TxGNN1.g_valid_pos, TxGNN1.g_valid_neg
# g_test_pos, g_test_neg = TxGNN1.g_test_pos, TxGNN1.g_test_neg

In [ ]:
# def load_pretrained(self, path, legacy=False):
#     with open(os.path.join(path, 'config.pkl'), 'rb') as f:
#         config = pickle.load(f)
        
#     self.model_initialize(**config, debug=True)
#     self.config = config
#     # self.G = initialize_node_embedding(self.G.to(torch.device("cpu")), config['n_inp']).to(self.device)
    
#     state_dict = torch.load(os.path.join(path, 'model.pt'), map_location = torch.device('cpu'))
#     if legacy: 
#         state_dict_G = torch.load(os.path.join(path, 'G.pt'), map_location = torch.device('cpu'))
#     else:
#         state_dict_G = torch.load(os.path.join(path, 'best_G.pt'), map_location = torch.device('cpu'))
#         # if next(iter(state_dict))[:7] == 'module.':
#         #     # the pretrained model is from data-parallel module
#         #     from collections import OrderedDict
#         #     new_state_dict = OrderedDict()
#         #     for k, v in state_dict.items():
#         #         name = k[7:] # remove `module.`
#         #         new_state_dict[name] = v
#         #     state_dict = new_state_dict
#         self.g_valid_pos = torch.load(os.path.join(path, 'g_valid_pos.pt'))
#         self.g_valid_neg = torch.load(os.path.join(path, 'g_valid_neg.pt'))
#         self.g_test_pos = torch.load(os.path.join(path, 'g_test_pos.pt'))
#         self.g_test_neg = torch.load(os.path.join(path, 'g_test_neg.pt'))
#     self.model.load_state_dict(state_dict)
#     for ntype, embs in state_dict_G.items():
#         self.G.nodes[ntype].data['inp'] = embs
#         if not legacy:
#             self.best_G.nodes[ntype].data['inp'] = embs

#     self.model = self.model.to(self.device)
#     self.G = self.G.to(self.device)
#     self.best_G = self.best_G.to(self.device)
#     self.best_model = self.model
#     print(f"Checkpoint loaded from path: {path}")

# load_pretrained(TxGNN1, save_dir, legacy=True)
# G = TxGNN1.G.to(TxGNN1.device)
# # best_G = TxGNN1.best_G.to(TxGNN1.device)
# best_model = TxGNN1.best_model.to(TxGNN1.device)
# best_model.eval()
# # print(TxGNN1.dd_etypes.device)
# # TxGNN1.model = TxGNN1.model.to(TxGNN1.device)
# # TxGNN1.best_model = TxGNN1.best_model.to(TxGNN1.device)
# # print(TxGNN1.model)
# # print("encore")

# ## Print out accuracies
# neg_sampler = Full_Graph_NegSampler(TxGNN1.G, 1, 'fix_dst', TxGNN1.device)
# g_train_neg = neg_sampler(TxGNN1.G)
# g_valid_pos, g_valid_neg = TxGNN1.g_valid_pos, TxGNN1.g_valid_neg
# g_test_pos, g_test_neg = TxGNN1.g_test_pos, TxGNN1.g_test_neg

In [ ]:
h_disease['disease_query_id']

In [ ]:
# mask = torch.isin(h_disease['disease_query_id'][0], h_disease['disease_key_id'][0])
# seen = mask.nonzero().squeeze() ## what does squeeze() do here? 
mask

In [ ]:
topk_values, topk_indices = torch.topk(sim[seen], self.k + 1, dim=1)

In [ ]:
embed = torch.zeros(h_disease['disease_query'].shape)
embed

In [ ]:
coef

In [ ]:
mask = torch.isin(h_disease['disease_query_id'][0], h_disease['disease_key_id'][0])
k=3

## any eval_g is in Train_G
coef = torch.zeros(sim.size(0), k, device=torch.device("cuda"))
embed = torch.zeros(h_disease['disease_query'].shape, device=torch.device("cuda"))
# seen_any = mask.nonzero().squeeze() > 0
seen = mask.nonzero().squeeze() ## what does squeeze() do here? 
sim = sim.to(torch.device("cuda"))
if len(seen) > 0:
    topk_values, topk_indices = torch.topk(sim[seen], k + 1, dim=1)
    coef[seen] = F.normalize(topk_values[:, 1:], p=1, dim=1)
    embed[seen] = h_disease['disease_key'][topk_indices[:, 1:]]

unseen = (~mask).nonzero().squeeze()
if len(unseen) > 0:
    topk_values, topk_indices = torch.topk(sim[unseen], k, dim=1)
    coef[unseen] = F.normalize(topk_values, p=1, dim=1)
    embed[unseen] = h_disease['disease_key'][topk_indices]
out = torch.mul(embed, coef.unsqueeze(dim = 2).to(torch.device("cuda"))).sum(dim = 1)

In [ ]:
# pd.read_csv
import pickle
with open("pseudo_valid_restrained_saveG.pkl", 'rb') as file:
    test = pickle.load(file)

test

In [ ]:
pred_score_pos.keys()

In [64]:
print(pred_score_pos[('drug', 'indication', 'disease')].sum())
print(len(pred_score_pos[('drug', 'indication', 'disease')]))
# print(pred_score_pos[('drug', 'contraindication', 'disease')].sum())

tensor(951.0067, device='cuda:0', grad_fn=<SumBackward0>)
1039


In [40]:
dd_df_valid[["x_idx", "y_idx"]]

,x_idx,y_idx
475906,885.0,12661.0
475908,301.0,12661.0
475909,7534.0,12661.0
475910,804.0,12661.0
475911,742.0,12661.0
...,...,...
480565,385.0,14500.0
480566,693.0,14500.0
480567,5786.0,15175.0
480568,187.0,13194.0


In [10]:
fname = '/home/euku/teams/b12/TxGNN/pseudo_valid_restrained_saveG.pkl'
with open(fname, "rb") as file:
    pseudo_valid = pickle.load(file)

# data_path = "./data/complex_disease_1/"
# df_valid = pd.read_csv(f"{data_path}valid.csv")
# dd_df_valid = df_valid[(df_valid.relation == "indication")]
# dd_df_valid

# rows = []
# for k, v in pseudo_valid["ranked_scores"].items():
#     for i, score in enumerate(v):
#         rows.append({'y_idx': k, 'score': score, 'x_idx': i})
# df_pseud_valid = pd.DataFrame(rows)
# df_pseud_valid

# df_pseud_valid['x_idx'] = df_pseud_valid['x_idx'].astype(float)
# df_pseud_valid['y_idx'] = df_pseud_valid['y_idx'].astype(float)

# temp = df_pseud_valid.merge(dd_df_valid[["x_idx", "y_idx"]], on=["x_idx", 'y_idx'], how='inner')
# temp

In [4]:
## How to extract the relation
with torch.no_grad():
    ## obtain the distmult, add an argument so I obtain the corresponding edge relation as well. 
    # h, beta_kl_loss, distmult = best_model(best_G, g_train_neg, pretrain_mode = False, mode = 'train', return_h_and_kl=True)
    h, beta_kl_loss, distmult = best_model(best_G, pretrain_mode = False, mode = 'train', return_h_and_kl=True)
    ## distmult params: (eval_G, G, h, pretrain_mode, mode = mode + '_pos', pseudo_training=pseudo_training)
    pred_score_pos, out_pos = distmult(g_valid_pos, best_G, h, pretrain_mode=False, mode='train_pos')
    # pred_score_neg, out_neg = distmult(g_valid_neg, best_G, h, pretrain_mode=False, mode='train_pos')
    # print(out_pos.sum())
    print(pred_score_pos[('drug', 'indication', 'disease')].sum())

    # pseudo_dd_etypes = [('drug', 'contraindication', 'disease'), 
    #                     ('drug', 'indication', 'disease'), ]
    # srcdst_dict = {f"{k}_pos": [] for k in pseudo_dd_etypes}
    # srcdst_dict.update({f"{k}_neg": [] for k in pseudo_dd_etypes})
    # for k in pseudo_dd_etypes:
    #     srcdst_dict[f"{k}_pos"] = pred_score_pos[f"{k}_srcdst"]
    #     srcdst_dict[f"{k}_neg"] = pred_score_pos[f"{k}_srcdst"]
    #     print(srcdst_dict[f"{k}_neg"])
    #     break

    # pred_score_pos, pred_score_neg, pos_score, neg_score, _ = model(G, g_neg, g_pos, pretrain_mode = False, mode = 'mode')
    
#     pos_score = torch.cat([pred_score_pos[i] for i in dd_etypes])
#     neg_score = torch.cat([pred_score_neg[i] for i in dd_etypes])
    
#     scores = torch.sigmoid(torch.cat((pos_score, neg_score)).reshape(-1,))
#     # labels = [1] * len(pos_score) + [0] * len(neg_score)
#     labels = torch.cat((torch.ones(len(pos_score), device=device),
#                         torch.zeros(len(neg_score), device=device)))
#     loss = F.binary_cross_entropy(scores, labels)

Using stored similarity matrix for ('drug', 'contraindication', 'disease')
Using stored similarity matrix for ('drug', 'indication', 'disease')
tensor(1102.1118, device='cuda:0')


In [9]:
pred_score_pos[('drug', 'indication', 'disease')]

tensor([ 0.0412,  0.0533,  0.0185,  ...,  0.0007, -0.0095,  0.0091],
       device='cuda:0')

In [72]:
shortened_pseudo_valid[shortened_pseudo_valid.y_idx == 15175.0]

,y_idx,score,x_idx


In [111]:
# pseudo_valid["ranked_scores"].keys()
dis_valid_ind[dis_valid_ind.y_idx == 15665.0]
# dis_valid_ind

,x_type,x_id,relation,y_type,y_id,x_idx,y_idx


In [7]:
fname = '/home/euku/teams/b12/TxGNN/pseudo_valid_restrained_saveG.pkl'
with open(fname, "rb") as file:
    pseudo_valid = pickle.load(file)

df_valid = TxGNN1.df_valid
# dis_idxs = [v for k,v in pseudo_valid['dis_idx'].items()]
dis_valid_ind = df_valid[df_valid.relation == "indication"]
dis_valid_idxs = df_valid[df_valid.relation == "indication"].y_idx
dis_idxs = [pseudo_valid['dis_idx'][k] for k in pseudo_valid["ranked_scores"].keys()]
mask = pd.Series(dis_idxs).isin(dis_valid_idxs)

# print(mask.sum())
print(pd.Series(dis_idxs)[mask])

rows = []
for i, (b, (k, dis_scores)) in enumerate(zip(mask.values, pseudo_valid["ranked_scores"].items())):
    if b:
        for drug_i, score in enumerate(dis_scores):
            rows.append({'y_idx': dis_idxs[i], 'score': score, 'x_idx': drug_i})

shortened_pseudo_valid = pd.DataFrame(rows)
shortened_pseudo_valid['y_idx'] = shortened_pseudo_valid['y_idx'].astype("float")
shortened_pseudo_valid['x_idx'] = shortened_pseudo_valid['x_idx'].astype("float")
shortened_pseudo_valid

shortened_pseudo_valid.merge(dis_valid_ind[["x_idx", "y_idx"]], on=['y_idx'], how="inner")
len(shortened_pseudo_valid.merge(dis_valid_ind[["x_idx", "y_idx"]], on=['y_idx'], how="inner").y_idx.unique())

shortened_pseudo_valid.merge(dis_valid_ind[["x_idx", "y_idx"]], on=['x_idx', 'y_idx'], how="inner").score.sum()

# sum([float(x) for x in pseudo_valid["ranked_scores"].keys()])
# sum([float(x) for x in pseudo_valid["ranked_scores"].keys()])
# [x for x in pseudo_valid["ranked_scores"].keys()]

19       7875.0
21      15045.0
26      12661.0
33      13149.0
40       5941.0
         ...   
5171      367.0
5173    14285.0
5174    12735.0
5175    14559.0
5176    15409.0
Length: 168, dtype: float64


y_idx    1.005916e+07
score    1.420248e+03
x_idx    2.293077e+06
dtype: float64

In [8]:
shortened_pseudo_valid.merge(dis_valid_ind[["x_idx", "y_idx"]], on=['x_idx', 'y_idx'], how="inner").score.sum()

1420.2483